In [ ]:
!pip install git+https://github.com/lesc-ufv/cad4u &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%load_ext plugin


Considere que desejamos gerar um array B através de um array A de mesmo tamanho, de modo que, cada elemento `B[i]` é dado pela soma do elemento `A[i]` mais os `r` elementos anteriores à posição `i` em A mais os `r` elementos posteriores, ou seja:
```
for(j=i-r; j<=i+r; j++)
    B[i]+=A[j]
```
O código dado a seguir já faz o cálculo do array B (armazenado em `b_cpu`) de forma sequencial. Escreva um kernel CUDA, bem como todo o restante do código necessário para executá-lo, de modo a permitir o cálculo de B de forma paralela.

As posições inválidas no início e final do array A devem ser simplesmente descartadas durante a soma, ou seja, para cálculo de `B[10]` com `r=32`, o resultado será a soma dos elementos de `A[0]` até `A[42]`.

Se atente para os seguinte requisitos:
* O cálculo dos elementos de B deve ser realizado de forma paralela;
* A memória compartilhada da GPU deve ser utilizada de forma a deixar a operação mais eficiente;
* O código deve tratar corretamente arrays grandes divididos em mais de um bloco.

Dicas:
* O código abaixo utiliza a constante `RANGE` para definir o valor de `r` e a constante `BLOCKSIZE` para definir o tamanho de cada bloco do grid;
* Por simplicidade, você pode considerar que o tamanho do array A será sempre múltiplo de `BLOCKSIZE`;
* Considere também que o tamanho de `r` será no mínimo igual a 0 e no máximo igual à metade de `BLOCKSIZE`, ou seja, o tamanho do array alocado na memória compartilhada será no máximo igual a `2*BLOCKSIZE`, e no mínimo igual ` BLOCKSIZE`.
* A estratégia utilizada pela versão sequencial de testar cada posição (para descobrir se é válida) não é ótima. Muitas comparações desnecessárias são realizadas ao calcular as posições intermediárias. Na versão paralela, é melhor preecher as posições inválidas com o valor 0 (zero) ao copiar os dados para a memória compartilhada.

In [ ]:
%%gpu
#include <stdio.h>
#include <cuda.h>
#include <chrono>  // Para medir o tempo na CPU

#define BLOCKSIZE 1024  // Threads por bloco
#define RANGE 256       // Deslocamento utilizado para cálculo de B

// Função para inicializar o array A
void initArray(int *a, int n) {
    for (int i = 0; i < n; ++i)
        a[i] = i;
}

// Função para imprimir o array
void printArray(int *a, int n) {
    int maxPrint = 20;  // Limita o número de elementos impressos
    for (int i = 0; i < std::min(n, maxPrint); ++i)
        printf("%5d ", a[i]);
    if (n > maxPrint) printf("... (array truncado)");
    printf("\n");
}

// Função para comparar dois arrays
bool checkArray(int *a, int *b, int n) {
    for (int i = 0; i < n; ++i)
        if (a[i] != b[i])
            return false;
    return true;
}

// Execução do cálculo na CPU
void execHost(int *A, int n, int *B) {
    for (int i = 0; i < n; i++) {
        int soma = 0;
        for (int j = i - RANGE; j <= i + RANGE; j++)
            if (j >= 0 && j < n)
                soma += A[j];
        B[i] = soma;
    }
}

// Execução do cálculo na GPU usando memória compartilhada
__global__
void execDevice(int* A, int *B, int n) {
    // Tamanho do array compartilhado (bloco + margens para o RANGE)
    int shared_SIZE = BLOCKSIZE + 2 * RANGE;
    __shared__ int sharedVec[BLOCKSIZE + 2 * RANGE];

    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x; // Índice global
    int localIdx = threadIdx.x; // Índice local no bloco

    // Índices inicial e final para carregar margens no array compartilhado
    int startIdx = blockIdx.x * blockDim.x - RANGE;
    int endIdx = (blockIdx.x + 1) * blockDim.x + RANGE - 1;

    // Carregar elementos no array compartilhado
    if (globalIdx >= 0 && globalIdx < n) {
        sharedVec[localIdx + RANGE] = A[globalIdx];
    } else {
        sharedVec[localIdx + RANGE] = 0;  // Fora do array, inicializa com 0
    }

    // Carregar margem esquerda
    if (globalIdx - RANGE < 0) {
        sharedVec[localIdx] = 0;
    } else {
        sharedVec[localIdx] = A[globalIdx - RANGE];
    }

    // Carregar margem direita
    if (globalIdx + RANGE >= n) {
        sharedVec[localIdx + RANGE + RANGE] = 0;
    } else {
        sharedVec[localIdx + RANGE + RANGE] = A[globalIdx + RANGE];
    }

    __syncthreads();  // Sincronização das threads do bloco

    // Cálculo da soma
    if (globalIdx >= 0 && globalIdx < n) {
        int soma = 0;
        for (int i = 0; i < (RANGE * 2) + 1; i++) {
            soma += sharedVec[localIdx + i];
        }
        B[globalIdx] = soma;
    }
}

int main() {
    int N = 1 << 20;              // Tamanho dos arrays A e B
    int *a = new int[N];
    int *b_cpu = new int[N];      // Array B calculado na CPU
    int *b_gpu = new int[N];      // Array B calculado na GPU

    // Inicializar array A
    initArray(a, N);
    printf("    A: ");
    printArray(a, N);

    // Alocação de memória na GPU
    int *d_a, *d_b;
    cudaError_t err;
    int size = N * sizeof(int);

    err = cudaMalloc((void**)&d_a, size);
    if (err != cudaSuccess) {
        printf("Erro na alocação de memória para A: %s\n", cudaGetErrorString(err));
        return -1;
    }

    err = cudaMalloc((void**)&d_b, size);
    if (err != cudaSuccess) {
        printf("Erro na alocação de memória para B: %s\n", cudaGetErrorString(err));
        return -1;
    }

    // Copiar array A para a GPU
    err = cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("Erro na cópia de memória de A para o dispositivo: %s\n", cudaGetErrorString(err));
        return -1;
    }

    // Medir tempo de execução na GPU
    int sharedMemSize = (BLOCKSIZE + 2 * RANGE) * sizeof(int);
    int numBlocks = (N + BLOCKSIZE - 1) / BLOCKSIZE;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    execDevice<<<numBlocks, BLOCKSIZE, sharedMemSize>>>(d_a, d_b, N);
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);


    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    // Copiar resultados da GPU para a CPU
    err = cudaMemcpy(b_gpu, d_b, size, cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
        printf("Erro na cópia de memória de B para a CPU: %s\n", cudaGetErrorString(err));
        return -1;
    }

    printf("B-GPU: ");
    printArray(b_gpu, N);

    // Medir tempo de execução na CPU
    auto startHost = std::chrono::high_resolution_clock::now();
    execHost(a, N, b_cpu);
    auto endHost = std::chrono::high_resolution_clock::now();
    auto durationHost = std::chrono::duration_cast<std::chrono::milliseconds>(endHost - startHost).count();


    printf("B-CPU: ");
    printArray(b_cpu, N);

    // Verificar se os resultados são iguais
    if (checkArray(b_gpu, b_cpu, N))
        printf("Resultados iguais\n");
    else
        printf("Resultados diferentes\n");



    printf("Tempo de execução na GPU: %.3f ms\n", milliseconds);
     printf("Tempo de execução na CPU: %lld ms\n", durationHost);
    // Liberar memória
    cudaFree(d_a);
    cudaFree(d_b);
    delete[] a;
    delete[] b_cpu;
    delete[] b_gpu;

    return 0;
}


    A:     0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15    16    17    18    19 ... (array truncado)
B-GPU: 32896 33153 33411 33670 33930 34191 34453 34716 34980 35245 35511 35778 36046 36315 36585 36856 37128 37401 37675 37950 ... (array truncado)
B-CPU: 32896 33153 33411 33670 33930 34191 34453 34716 34980 35245 35511 35778 36046 36315 36585 36856 37128 37401 37675 37950 ... (array truncado)
Resultados iguais
Tempo de execução na GPU: 1.640 ms
Tempo de execução na CPU: 1538 ms

